In [1]:
import pandas as pd

## Data

In [2]:
import json

def read_json(file):
    with open(file) as f:
        return json.load(f)


In [3]:
# df = pd.read_csv('eval_scenarios/reception_agent_adapt.csv',delimiter=";")
df = pd.read_csv('outputs/try_agent_reception_20241011-141555.csv')
conditions = read_json('eval_scenarios/conditions.json')
goals = read_json('eval_scenarios/goals.json')

In [4]:
df.columns

Index(['num_condition', 'type', 'goals', 'GT_subgoals_disfavour',
       'GT_subgoals_favour', 'agent', 'subgoals_disfavour', 'subgoals_favour'],
      dtype='object')

In [5]:
df['condition'] = df['num_condition'].apply(lambda x: conditions[str(x)]['condition'])
df['disfavour_condition'] = df['num_condition'].apply(lambda x: conditions[str(x)]['disfavour'])

In [6]:
# Función para seleccionar los subgoals correctos a partir de múltiples goals en una fila
def get_subgoals(row, conditions):
    condition = row['condition']
    goal_ids = eval(row['goals'])  # Suponemos que los ids de los goals están separados por comas
    subgoals_list = []
    # Iterar sobre todos los goal_ids y obtener los subgoals correspondientes
    for goal_id in goal_ids:
        goal_id = goal_id.strip()  # Eliminar espacios en blanco
        if 'human' in condition:
            subgoals_list.extend(goals[goal_id]['subgoals_human'])
        elif 'robot' in condition:
            subgoals_list.extend(goals[goal_id]['subgoals_robot'])
    
    return subgoals_list

# Crear la nueva columna con los subgoals concatenados en una lista
df['subgoals'] = df.apply(lambda row: get_subgoals(row, conditions), axis=1)

## Execute LLM

In [7]:
import openai
import os

from llm_env_to_goal import LLMEnvToGoalReasoner
from llm_agent_to_action import LLMAgentToActionAllocationReasoner

from dotenv import load_dotenv

In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")
llm_agent_to_action = LLMAgentToActionAllocationReasoner("prompts")

def apply_llm_action_costs(row):
    # Llamar a la función con los subgoals y condition de la fila actual
    agent, subgoals_disfavour, subgoals_favour, pddl_costs = llm_agent_to_action.llm_pddl_action_costs_for_agent_condition(
        str(row['subgoals']), row['condition']
    )
    
    # Devolver los valores en un diccionario para agregarlos como nuevas columnas
    return pd.Series({
        'agent': agent,
        'subgoals_disfavour': subgoals_disfavour,
        'subgoals_favour': subgoals_favour,
        'pddl_costs': pddl_costs
    })


In [18]:
subgoals = df['subgoals'][0]

In [19]:
subgoals

['carta colocada sobre',
 'sello puesto sobre',
 'sobre enviado correus',
 'paquete recogido loc2_entradaFME',
 'registros colocados caja',
 'caja llevada almacen']

In [16]:
condition = df['condition'][2]

In [17]:
condition

'El robot no puede recoger el paquete porque un humano debe firmarlo.'

In [21]:
agent, subgoals_disfavour, subgoals_favour, pddl_costs = llm_agent_to_action.llm_pddl_action_costs_for_agent_condition(str(subgoals), condition)

El agente deberia desfavorecer estas tareas:
paquete recogido loc2_entradaFME

El agente deberia favorecer estas tareas:
Ninguna
['paquete recogido loc2_entradaFME']
['paquete recogido loc2_entradaFME']
['paquete recogido loc2_entradaFME', '']
['paquete recogido loc2_entradaFME']
['']


In [22]:
subgoals_disfavour

['paquete recogido loc2_entradaFME']

In [41]:
def apply_llm_action_costs(row):
    try:
        agent, subgoals_disfavour, subgoals_favour, pddl_costs = llm_agent_to_action.llm_pddl_action_costs_for_agent_condition(
            str(row['subgoals']), row['condition']
        )
        return pd.Series({
            'agent': agent,
            'subgoals_disfavour': subgoals_disfavour,
            'subgoals_favour': subgoals_favour,
            'pddl_costs': pddl_costs
        })
    except Exception as e:
        print(f"Error processing row: {row}")
        print(f"Error message: {str(e)}")
        return pd.Series({'agent': None, 'subgoals_disfavour': None, 'subgoals_favour': None, 'pddl_costs': None})

# Apply the function and handle potential errors
result = df.apply(apply_llm_action_costs, axis=1)
print(f"Shape of result: {result.shape}")
print(f"Columns in result: {result.columns}")

# Assign the results to the DataFrame, ignoring any rows that caused errors
df[['agent', 'subgoals_disfavour', 'subgoals_favour', 'pddl_costs']] = result


El agente deberia desfavorecer estas tareas:
Ninguna

El agente deberia favorecer estas tareas:
['carta colocada sobre', 'sello puesto sobre', 'sobre enviado correus', 'paquete recogido loc2_entradaFME', 'registros colocados caja', 'caja llevada almacen']
[]
['', "['carta colocada sobre', 'sello puesto sobre', 'sobre enviado correus', 'paquete recogido loc2_entradaFME', 'registros colocados caja', 'caja llevada almacen']"]
['']
["['carta colocada sobre'", "'sello puesto sobre'", "'sobre enviado correus'", "'paquete recogido loc2_entradaFME'", "'registros colocados caja'", "'caja llevada almacen']"]
{'attribute': "'sello puesto sobre'_cost", 'agent': 'human', 'obj': "['carta colocada sobre'", 'loc': "'sobre enviado correus'", 'value': 100}
El agente deberia desfavorecer estas tareas:
Ninguna

El agente deberia favorecer estas tareas:
['carta colocada sobre', 'sello puesto sobre', 'sobre enviado correus', 'carrito recogido loc3', 'paquete recogido loc2_entradaFME', 'carrito dejado loc3',

## Add GT and calculate

In [43]:
df['GT_subgoals_disfavour'] = df.apply(lambda row: list(set(row['subgoals']).intersection(row['disfavour_condition'])), axis=1)

In [44]:
df.drop(columns=['condition','subgoals','disfavour_condition'],inplace=True)

In [45]:
df['GT_subgoals_disfavour'].values[2]

['paquete recogido loc2_entradaFME']

In [49]:
# df['subgoals_disfavour'] = df['subgoals_disfavour'].apply(lambda x: eval(x))

In [47]:

# Función para calcular Completeness y Correctness
def calculate_completeness_correctness(row):
    gt_set = set(row['GT_subgoals_disfavour'])
    subgoal_set = set(row['subgoals_disfavour'])
    
    if len(gt_set) == 0:
        completeness = 1.0 if len(subgoal_set) == 0 else 0.0
    else:
        completeness = len(subgoal_set & gt_set) / len(gt_set)
    
    if len(subgoal_set) == 0:
        correctness = 1.0 if len(gt_set) == 0 else 0.0
    else:
        correctness = len(subgoal_set & gt_set) / len(subgoal_set)
    
    return pd.Series([completeness, correctness])

# Aplicar la función al DataFrame
df[['Completeness', 'Correctness']] = df.apply(calculate_completeness_correctness, axis=1)


In [48]:
df.describe()

,num_condition,Completeness,Correctness
count,27.000000,27.000000,27.000000
mean,5.000000,0.722222,0.677160
std,2.631174,0.445778,0.452891
min,1.000000,0.000000,0.000000
25%,3.000000,0.250000,0.100000
50%,5.000000,1.000000,1.000000
75%,7.000000,1.000000,1.000000
max,9.000000,1.000000,1.000000


In [53]:
df[(df['Completeness']==1)&(df['Correctness']!=1)]

,num_condition,type,goals,GT_subgoals_disfavour,GT_subgoals_favour,agent,subgoals_disfavour,subgoals_favour,pddl_costs,Completeness,Correctness
8,9,1 - Robot negative,"['1','3','5']","[carta colocada sobre, sello puesto sobre]",none,human,"[carta colocada sobre, sello puesto sobre, sob...","[carta colocada sobre, sello puesto sobre, sob...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {...",1.0,0.333333
13,5,1 - Robot negative,"['1','2','7']",[sobre enviado correus],none,robot,"[carta colocada sobre, sello puesto sobre, sob...","[carta colocada sobre, sello puesto sobre, sob...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]",1.0,0.200000
17,9,1 - Robot negative,"['1','2','7']","[carta colocada sobre, diplomas sellados loc1,...",none,human,"[carta colocada sobre, sello puesto sobre, sob...","[carta colocada sobre, sello puesto sobre, sob...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]",1.0,0.750000


In [58]:
from datetime import datetime
today = datetime.now().strftime("%Y%m%d-%H%M%S")
df.to_csv(f'outputs/try_agent_reception_ESP_{today}.csv',index=False)